# 미리 학습된 모델에 이미지와 동영상을 넣어 output 만들기

### Step 1. 모델 정의

In [ ]:
import torch
import torch.nn as nn

class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()

        def CBR(in_channels, out_channels):
            layers = [
                nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(inplace=True)
            ]
            return nn.Sequential(*layers)

        # Contracting path
        self.enc1 = nn.Sequential(CBR(1, 64), CBR(64, 64))
        self.pool1 = nn.MaxPool2d(2)

        self.enc2 = nn.Sequential(CBR(64, 128), CBR(128, 128))
        self.pool2 = nn.MaxPool2d(2)

        self.enc3 = nn.Sequential(CBR(128, 256), CBR(256, 256))
        self.pool3 = nn.MaxPool2d(2)

        self.enc4 = nn.Sequential(CBR(256, 512), CBR(512, 512))
        self.pool4 = nn.MaxPool2d(2)

        self.center = CBR(512, 1024)

        # Expansive path
        self.up4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.dec4 = nn.Sequential(CBR(1024, 512), CBR(512, 512))

        self.up3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec3 = nn.Sequential(CBR(512, 256), CBR(256, 256))

        self.up2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec2 = nn.Sequential(CBR(256, 128), CBR(128, 128))

        self.up1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec1 = nn.Sequential(CBR(128, 64), CBR(64, 64))

        #self.final = nn.Conv2d(64, 1, kernel_size=1)
        #클래스 수 변경하기 epochs5 이후에 음수 validation loss 값이 나와서 변경
        self.final = nn.Conv2d(64, 3, kernel_size=1)  # 3은 클래스 수

    def forward(self, x):
      # Encoder
      enc1 = self.enc1(x)
      pool1 = self.pool1(enc1)

      enc2 = self.enc2(pool1)
      pool2 = self.pool2(enc2)

      enc3 = self.enc3(pool2)
      pool3 = self.pool3(enc3)

      enc4 = self.enc4(pool3)
      pool4 = self.pool4(enc4)

      # Center
      center = self.center(pool4)

      # Decoder
      up4 = self.up4(center)

      # 크기를 맞추기 위해 자르기 (업샘플링 후 텐서 크기 불일치 문제 해결)
      if up4.size() != enc4.size():
          diffY = enc4.size(2) - up4.size(2)
          diffX = enc4.size(3) - up4.size(3)
          up4 = torch.nn.functional.pad(up4, [0, diffX, 0, diffY])

      merge4 = torch.cat([up4, enc4], dim=1)
      dec4 = self.dec4(merge4)

      up3 = self.up3(dec4)
      if up3.size() != enc3.size():
          diffY = enc3.size(2) - up3.size(2)
          diffX = enc3.size(3) - up3.size(3)
          up3 = torch.nn.functional.pad(up3, [0, diffX, 0, diffY])

      merge3 = torch.cat([up3, enc3], dim=1)
      dec3 = self.dec3(merge3)

      up2 = self.up2(dec3)
      if up2.size() != enc2.size():
          diffY = enc2.size(2) - up2.size(2)
          diffX = enc2.size(3) - up2.size(3)
          up2 = torch.nn.functional.pad(up2, [0, diffX, 0, diffY])

      merge2 = torch.cat([up2, enc2], dim=1)
      dec2 = self.dec2(merge2)

      up1 = self.up1(dec2)
      if up1.size() != enc1.size():
          diffY = enc1.size(2) - up1.size(2)
          diffX = enc1.size(3) - up1.size(3)
          up1 = torch.nn.functional.pad(up1, [0, diffX, 0, diffY])

      merge1 = torch.cat([up1, enc1], dim=1)
      dec1 = self.dec1(merge1)

      out = self.final(dec1)
      return out

### Step 2. 데이터셋 클래스 및 Transform 정의

PyTorch의 Dataset과 DataLoader를 사용하여 데이터를 불러옵니다.

In [ ]:
import os
import json
import numpy as np
import cv2
import torch
from torch.utils.data import Dataset

class PS_Dataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.input_paths = sorted([os.path.join(data_dir, 'images', f) for f in os.listdir(os.path.join(data_dir, 'images')) if f.endswith('.jpg')])
        self.label_paths = sorted([os.path.join(data_dir, 'labels', f) for f in os.listdir(os.path.join(data_dir, 'labels')) if f.endswith('.json')])

    def __len__(self):
        return len(self.input_paths)

    def __getitem__(self, idx):
        # 이미지 로드 (jpg 파일)
        input_image = cv2.imread(self.input_paths[idx], cv2.IMREAD_GRAYSCALE)  # 흑백으로 읽음
        input_image = input_image / 255.0  # 정규화 [0, 1]
        input_image = np.expand_dims(input_image, axis=0).astype(np.float32)  # 채널 차원 추가 (C, H, W)

        # 라벨 로드 (json 파일)
        with open(self.label_paths[idx], 'r') as f:
            label_data = json.load(f)

        # JSON 데이터를 기반으로 다중 클래스 레이블 생성
        label_image = np.zeros((input_image.shape[1], input_image.shape[2]), dtype=np.uint8)  # (H, W) 크기

        for region in label_data['segmentation']:
            label_name = region['name']  # 클래스 이름: "Driveable Space" 또는 "Parking Space"
            points = np.array(region['polygon'], dtype=np.int32)  # 폴리곤 좌표

            # 클래스 할당 (0: Background, 1: Driveable Space, 2: Parking Space)
            if label_name == 'Driveable Space':
                cv2.fillPoly(label_image, [points], 1)  # Driveable Space -> 클래스 1
            elif label_name == 'Parking Space':
                cv2.fillPoly(label_image, [points], 2)  # Parking Space -> 클래스 2

        label_image = label_image.astype(np.float32)  # Float 변환
        label_image = np.expand_dims(label_image, axis=0)  # (C, H, W) 차원 추가

        # Tensor로 변환
        input_image = torch.from_numpy(input_image)
        label_image = torch.from_numpy(label_image)

        # Transform 적용 (필요한 경우)
        if self.transform:
            input_image = self.transform(input_image)

        return {'input': input_image, 'label': label_image}

#### 3.1 Transform 정의

In [ ]:
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Normalize(mean=[0.5], std=[0.5])
])

###### 학습된 모델을 불러와서 성능 검증하기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


- 동영상을 넣어서 성능 검증하기

In [ ]:
import cv2
import numpy as np
import torch

# 모델 경로
model_path = '/content/drive/MyDrive/likelion_CV/segmentation_project/unet_model50.pth'

# 동영상 파일 경로
video_path = '/content/drive/MyDrive/likelion_CV/segmentation_project/parkingspace_video(640,360).mp4'

# 결과 저장 경로
output_video_path = '/content/drive/MyDrive/likelion_CV/segmentation_project/parkingspace_video_output50.mp4'

# 색상 맵핑 설정
segmentation_colors = {
    0: (0, 0, 0),       # Background
    1: (0, 255, 0),     # Driveable Space
    2: (255, 0, 0)      # Parking Space
}

# GPU/CPU 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델 초기화 및 가중치 불러오기
model = UNet()  # UNet 클래스 정의 필요
model.load_state_dict(torch.load(model_path, map_location=device))  # 가중치 로드
model.eval()  # 평가 모드 설정
model.to(device)  # 모델을 GPU/CPU로 이동

# 동영상 로드
cap = cv2.VideoCapture(video_path)

# 동영상 정보 가져오기
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# 출력 동영상 코덱 설정 (MP4V 코덱 사용)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# 프레임 단위로 동영상 처리
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 프레임 전처리 (모델 입력 크기에 맞게 리사이즈 및 그레이스케일 변환)
    input_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # 컬러 이미지를 그레이스케일로 변환
    input_frame_resized = cv2.resize(input_frame, (640, 360))  # 모델 크기에 맞게 리사이즈
    input_tensor = torch.from_numpy(input_frame_resized).unsqueeze(0).unsqueeze(0).float() / 255.0  # 1채널로 변경
    input_tensor = input_tensor.to(device)

    # 모델 예측
    with torch.no_grad():
        outputs = model(input_tensor)
        outputs = torch.softmax(outputs, dim=1)
        predicted_classes = torch.argmax(outputs, dim=1).cpu().numpy().squeeze()

    # 세그멘테이션 결과를 컬러 이미지로 변환
    output_colored = np.zeros((*predicted_classes.shape, 3), dtype=np.uint8)
    for class_id, color in segmentation_colors.items():
        output_colored[predicted_classes == class_id] = color

    # 결과 이미지 원본 크기로 리사이즈
    output_colored = cv2.resize(output_colored, (width, height))

    # 결과 이미지와 원본 이미지 합성
    overlay = cv2.addWeighted(frame, 0.6, output_colored, 0.4, 0)

    # 결과 동영상 저장
    out.write(overlay)

# 모든 작업 완료 후 자원 해제
cap.release()
out.release()
print(f"처리가 완료되었습니다. 결과 동영상은 {output_video_path}에 저장되었습니다.")

<ipython-input-5-bccebb679c47>:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))  # 가중치 로드


처리가 완료되었습니다. 결과 동영상은 /content/drive/MyDrive/likelion_CV/segmentation_project/parkingspace_video_output50.mp4에 저장되었습니다.


- 이미지를 세그멘테이션으로 만든 후에 동영상으로 변환

In [ ]:
import os
import torch
import cv2
import numpy as np
from torch import nn

# 모델 경로
model_path = '/content/drive/MyDrive/likelion_CV/segmentation_project/unet_model50.pth'

# 출력 결과를 저장할 폴더
output_folder = '/content/drive/MyDrive/likelion_CV/segmentation_project/segmentation_results2'

# 출력 폴더가 존재하지 않으면 생성
if not os.path.exists(output_folder):
    os.makedirs(output_folder)


# 모델 초기화 및 가중치 불러오기
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet()
model.load_state_dict(torch.load(model_path, map_location=device), strict=False)
model.to(device)
model.eval()

# 색상 맵핑 설정
segmentation_colors = {
    0: (0, 0, 0),       # Background (검정색)
    1: (0, 255, 0),     # Driveable Space (녹색)
    2: (255, 0, 0)      # Parking Space (빨간색)
}

# 입력 이미지 폴더 경로
image_folder = '/content/drive/MyDrive/likelion_CV/segmentation_project/im2'

# 폴더 내 모든 이미지 처리
for image_name in os.listdir(image_folder):
    image_path = os.path.join(image_folder, image_name)

    # 이미지 로드 및 전처리
    image = cv2.imread(image_path)
    if image is None:
        print(f"이미지를 불러올 수 없습니다: {image_name}")
        continue

    original_height, original_width = image.shape[:2]
    image_resized = cv2.resize(image, (640, 360))  # 모델 입력 크기에 맞게 리사이즈
    input_frame = cv2.cvtColor(image_resized, cv2.COLOR_BGR2GRAY)  # 그레이스케일 변환
    input_tensor = torch.from_numpy(input_frame).unsqueeze(0).unsqueeze(0).float().to(device) / 255.0  # 정규화 및 차원 추가

    # 모델 예측
    with torch.no_grad():
        output = model(input_tensor)
        predicted_classes = torch.argmax(output, dim=1).squeeze().cpu().numpy()

    # 세그멘테이션 결과를 컬러로 변환
    output_colored = np.zeros((*predicted_classes.shape, 3), dtype=np.uint8)
    for class_id, color in segmentation_colors.items():
        output_colored[predicted_classes == class_id] = color

    # 원본 이미지 크기로 리사이즈
    output_colored = cv2.resize(output_colored, (original_width, original_height))

    # 원본 이미지와 세그멘테이션 결과를 오버레이
    overlay = cv2.addWeighted(image, 0.6, output_colored, 0.4, 0)

    # 결과 저장
    output_image_path = os.path.join(output_folder, f"overlay_{image_name}")
    cv2.imwrite(output_image_path, overlay)
    #print(f"오버레이 세그멘테이션 결과 저장: {output_image_path}")

<ipython-input-24-99bdde366b27>:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device), strict=False)


In [ ]:
import os
import cv2

# 세그멘테이션 결과 이미지 폴더 경로
segmentation_folder = '/content/drive/MyDrive/likelion_CV/segmentation_project/segmentation_results2'

# 출력 동영상 경로
output_video_path = '/content/drive/MyDrive/likelion_CV/segmentation_project/segmentation_imtovi_output.mp4'

# 이미지 파일 리스트 정렬
image_files = sorted(os.listdir(segmentation_folder))

# 동영상 저장 설정
fps = 10  # 초당 프레임 수 (FPS)
frame_size = None  # 프레임 크기 (첫 번째 이미지에서 가져올 예정)

# VideoWriter 초기화
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # MP4V 코덱
video_writer = None

# 이미지 순서대로 읽어서 동영상 생성
for image_file in image_files:
    image_path = os.path.join(segmentation_folder, image_file)
    frame = cv2.imread(image_path)

    if frame is None:
        print(f"이미지를 불러올 수 없습니다: {image_file}")
        continue

    if frame_size is None:
        # 첫 번째 프레임 크기로 설정
        frame_size = (frame.shape[1], frame.shape[0])  # (width, height)
        video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, frame_size)

    video_writer.write(frame)

# VideoWriter 자원 해제
if video_writer is not None:
    video_writer.release()

print(f"영상 생성 완료: {output_video_path}")

영상 생성 완료: /content/drive/MyDrive/likelion_CV/segmentation_project/segmentation_video_output.mp4
